## Import Library

In [ ]:
import folium
from folium import plugins
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly_express as px
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

## Preview Data

In [ ]:
df = pd.read_csv('../input/birdclef-2021/train_metadata.csv')
df.head()

## Check unique primary label

In [ ]:
print('unique primary label : ', len(df['primary_label'].unique()),'\n',df['primary_label'].unique())

## Bird locoation visualization (season)

* Hover on the top-right button to open label list
* 4 color corresponding to 4 seasons
 🟢 Sprint
 🟡 Summer
 🟠 Fall
 🔵 Winter
* Click color circle to check information (primary label、date、season)

## Observation results

As we can see, most of bird will move to south in winter and move back to north in summer time

In [ ]:
bird_df = df[df['primary_label'] == 'houwre']

In [ ]:
route_df = bird_df[['primary_label', 'latitude', 'longitude', 'date']]
route_df = route_df.sort_values(by='date',ascending=True)

# Filter out rows with wrong format date
route_df = route_df[route_df.date.str.split('-').str[2] != '00']
route_df = route_df[route_df.date.str.split('-').str[1] != '00']
route_df = route_df[route_df.date.str.split('-').str[0].astype(int) > 1000]

route_df['season'] = pd.to_datetime(route_df['date'], format='%Y-%m-%d').dt.month%12 // 3 + 1

mapa = folium.Map(height = 500, width = 1000, location=[route_df.iloc[0]['latitude'], route_df.iloc[0]['longitude']],zoom_start=4,tiles="cartodbpositron")

for grp_name, df_grp in route_df.groupby('primary_label'):
    feature_group = folium.FeatureGroup(grp_name, show=True)
    for row in df_grp.itertuples():
        if row.season == 1:
            color = 'blue'
            season = 'winter'
        elif row.season == 2:
            color = 'green'
            season = 'spring'
        elif row.season == 3:
            color = 'yellow'
            season = 'summer'
        else:
            color = 'orange'
            season = 'fall'

        folium.Circle(
            radius=40000,
            location=[row.latitude, row.longitude],
            popup= f"{row.primary_label}_ \n {row.date}_ \n{season}",
            color = "#555",
            weight=2,
            fill_color = color,
            fill_opacity=1,
        ).add_to(feature_group)
        
    feature_group.add_to(mapa)

folium.LayerControl().add_to(mapa)
mapa